# Project 11 -- Anthony Micco

**TA Help:** N/A

**Collaboration:** N/A

**Internet Resources:** Used Stack Overflow blog to help extract month out of the data variable for question 4.4 (https://stackoverflow.com/questions/26105804/extract-month-from-date-in-python) 

**ChatGPT, Gemini, Claude, etc:** None

***Link to AI Chat History***: None

**OVERALL MESSAGE:** Any time that you used anything except your brain to solve the questions in these projects, you need to disclose such resources at the start of the project, with details about your usage of the tools.

**YOUR OWN WORK:** Even when you utilize other resources, do NOT just copy and paste.  Write all explanations in your own words, using several sentences in English, which are understandable and which you wrote (and did not just copy and paste).

## Question 1

In [26]:
# adding our imports
from pymongo import MongoClient
import pandas as pd
import json

In [27]:
#defining our MongoClient, database and collections
client = MongoClient('mongodb://localhost:27017/')

db = client['ecommerce_db']

products = db['products']
customers = db['customers']
orders = db['orders']

In [28]:
#loading the ecommerce dataset into the monogdb database we created

# Load the e-commerce dataset from JSON file
with open('/anvil/projects/tdm/data/ecommerce/fake_ecommerce_dataset.json', 'r') as file:
    dataset = json.load(file)

# Insert the data into MongoDB
products.insert_many(dataset['products'])
customers.insert_many(dataset['customers'])
orders.insert_many(dataset['orders'])

print("Dataset loaded successfully!")

Dataset loaded successfully!


In [29]:
#examining a single sample product from the database
sample_product = products.find_one()
print("Sample Product Structure:")
print(f"Product Name: {sample_product['name']}")
print(f"Category: {sample_product['category']}")
print(f"Price: ${sample_product['price']}")
print(f"Specifications: {sample_product['specifications']}")
print(f"Number of Reviews: {len(sample_product['reviews'])}")
print(f"Tags: {sample_product['tags']}")

Sample Product Structure:
Product Name: Wireless Headphones
Category: Electronics
Price: $99.99
Specifications: {'brand': 'TechSound', 'color': 'Black', 'battery_life': '20 hours', 'connectivity': 'Bluetooth 5.0', 'weight': '250g', 'warranty': '2 years'}
Number of Reviews: 3
Tags: ['audio', 'wireless', 'bluetooth', 'headphones']


In [30]:
#using our dataset to perform basic queries

# Find products in a specific category
electronics = list(products.find({"category": "Electronics"}))
print(f"Electronics products: {len(electronics)}")

# Find products with price greater than $200
expensive_products = list(products.find({"price": {"$gt": 200}}))
print(f"Products over $200: {len(expensive_products)}")

# Find customers from a specific state
ca_customers = list(customers.find({"address.state": "CA"}))
print(f"Customers from California: {len(ca_customers)}")

# Find orders with specific status
completed_orders = list(orders.find({"status": "completed"}))
print(f"Completed orders: {len(completed_orders)}")

Electronics products: 48
Products over $200: 32
Customers from California: 24
Completed orders: 40


1.9) Dot notation for querying nest fields in MongoDB works very similar to how dictionaries work in Python. You use the parent object's name and then to access any nested data you put dot the field you want. For example if we have an object called player and we want to access the player's name we would access this by doing player.name to get the player's name. 

## Question 2

In [31]:
# practicing using $elemMatch with nested queries

# Query nested fields using dot notation
ca_customers = list(customers.find({"address.state": "CA"}))
print(f"Customers from California: {len(ca_customers)}")

# Query arrays using $elemMatch
five_star_products = list(products.find({
    "reviews": {"$elemMatch": {"rating": 5}}
}))
print(f"Products with 5-star reviews: {len(five_star_products)}")

# Combine multiple conditions
high_priced_electronics = list(products.find({
    "$and": [
        {"category": "Electronics"},
        {"price": {"$gt": 100}}
    ]
}))
print(f"High priced electronic products: {len(high_priced_electronics)}")

Customers from California: 24
Products with 5-star reviews: 120
High priced electronic products: 32


In [32]:
#query to find 5-star reviews using $elemMatch

fiveStarProducts = list(products.find({
    "reviews": {"$elemMatch" : {"rating":5}}
}))
print(f"Products with 5 star reviews: {len(fiveStarProducts)}")

Products with 5 star reviews: 120


In [33]:
#finding the order field structure using algorithm from project 10
sample_order = orders.find_one()
print("Product document structure:")
for key, value in sample_order.items():
    print(f"{key}: {type(value).__name__}")
    if isinstance(value, str) and len(str(value)) > 100:
        print(f"  Value: {str(value)[:100]}...")
    else:
        print(f"  Value: {value}")
    print()

Product document structure:
_id: ObjectId
  Value: 690d0246b935a0b6f4811656

order_id: str
  Value: O001

customer_id: str
  Value: C001

order_date: str
  Value: 2024-01-15T10:30:00Z

status: str
  Value: completed

total_amount: float
  Value: 229.98

shipping_address: dict
  Value: {'street': '123 Main St', 'city': 'New York', 'state': 'NY', 'zip': '10001', 'country': 'USA'}

items: list
  Value: [{'product_id': 'P001', 'quantity': 1, 'price': 99.99, 'subtotal': 99.99}, {'product_id': 'P003', 'quantity': 1, 'price': 129.99, 'subtotal': 129.99}]

payment_method: str
  Value: credit_card

shipping_cost: float
  Value: 9.99

tax: float
  Value: 18.0



In [34]:
#finding the customer field structure using algorithm from project 10
sample_customer = customers.find_one()
print("Product document structure:")
for key, value in sample_customer.items():
    print(f"{key}: {type(value).__name__}")
    if isinstance(value, str) and len(str(value)) > 100:
        print(f"  Value: {str(value)[:100]}...")
    else:
        print(f"  Value: {value}")
    print()

Product document structure:
_id: ObjectId
  Value: 690d0246b935a0b6f4811647

customer_id: str
  Value: C001

name: str
  Value: John Doe

email: str
  Value: john@email.com

phone: str
  Value: 555-0101

address: dict
  Value: {'street': '123 Main St', 'city': 'New York', 'state': 'NY', 'zip': '10001', 'country': 'USA'}

preferences: dict
  Value: {'categories': ['Electronics', 'Sports'], 'newsletter': True, 'notifications': True, 'language': 'English'}

created_at: str
  Value: 2024-01-01T00:00:00Z



In [35]:
#finding the product review user field structure using algorithm from project 10
sample_product = products.find_one()
print("Product document structure:")
for key, value in sample_product.items():
    print(f"{key}: {type(value).__name__}")
    if isinstance(value, str) and len(str(value)) > 100:
        print(f"  Value: {str(value)[:100]}...")
    else:
        print(f"  Value: {value}")
    print()

Product document structure:
_id: ObjectId
  Value: 690d0246b935a0b6f4811638

product_id: str
  Value: P001

name: str
  Value: Wireless Headphones

category: str
  Value: Electronics

price: float
  Value: 99.99

stock: int
  Value: 50

specifications: dict
  Value: {'brand': 'TechSound', 'color': 'Black', 'battery_life': '20 hours', 'connectivity': 'Bluetooth 5.0', 'weight': '250g', 'warranty': '2 years'}

reviews: list
  Value: [{'rating': 5, 'comment': 'Great sound quality!', 'user': 'john_doe', 'date': '2024-01-15'}, {'rating': 4, 'comment': 'Good value for money', 'user': 'jane_smith', 'date': '2024-01-20'}, {'rating': 5, 'comment': 'Excellent noise cancellation', 'user': 'audio_fan', 'date': '2024-02-01'}]

tags: list
  Value: ['audio', 'wireless', 'bluetooth', 'headphones']

created_at: str
  Value: 2024-01-01T00:00:00Z



In [36]:
#query to find a review from a specific user

userReviews = list(products.find({
    "reviews":{"$elemMatch":{"user":"john_doe"}}
}))
print(f"Reviews by john_doe: {len(userReviews)}")

Reviews by john_doe: 8


In [37]:
#query to find products with reviews with a rating of >= 4 and contain the word excellent in the comment

excellentRatings = list(products.find({
    "reviews":{"$elemMatch":{"rating": {"$gte" : 4}, "comment":{"$regex": "Excellent"}}}
}))
print(f"Reviews with rating >= 4 and containing the word excellent: {len(excellentRatings)}")

Reviews with rating >= 4 and containing the word excellent: 24


2.8) When querying arrays without \\$elemMatch, the query returns any element that has the criteria you are searching for. This means that if you are querying an array for a review greater than 4 and containing the word excellent in the comment, it will return ALL the products that have either of those characteristics, regardless if a single product does not satisfy both criteria. However, if you use \\$elemMatch, the criteria you query on will force the query to return every element that satisfies all the criteria you are filtering meaning a single product must have both a review greater than 4 AND its comment should contain the word excellent.

## Question 3

In [38]:
#practicing with the example queries

# Calculate average rating for each product using $unwind
pipeline = [
    {"$unwind": "$reviews"},  # Create one document per review
    {"$group": {
        "_id": "$product_id",
        "product_name": {"$first": "$name"},
        "avg_rating": {"$avg": "$reviews.rating"},
        "review_count": {"$sum": 1}
    }},
    {"$sort": {"avg_rating": -1}}
]

top_rated = list(products.aggregate(pipeline))
print("Top rated products:")
for product in top_rated[:3]:  # Show top 3
    print(f"{product['product_name']}: {product['avg_rating']:.2f}")

Top rated products:
Gaming Mouse: 4.67
Laptop: 4.67
Air Fryer: 4.67


In [39]:
#practice continued

# Count products by category with average price
category_pipeline = [
    {"$group": {
        "_id": "$category",
        "count": {"$sum": 1},
        "avg_price": {"$avg": "$price"},
        "total_stock": {"$sum": "$stock"}
    }},
    {"$sort": {"count": -1}}
]

category_stats = list(products.aggregate(category_pipeline))
print("Products by category:")
for category in category_stats:
    print(f"{category['_id']}: {category['count']} products, avg price: ${category['avg_price']:.2f}, total stock: {category['total_stock']}")

# Find customers by state with newsletter subscription rates
state_pipeline = [
    {"$group": {
        "_id": "$address.state",
        "customer_count": {"$sum": 1},
        "newsletter_subscribers": {
            "$sum": {"$cond": [{"$eq": ["$preferences.newsletter", True]}, 1, 0]}
        }
    }},
    {"$sort": {"customer_count": -1}}
]

state_stats = list(customers.aggregate(state_pipeline))
print("\nCustomers by state:")
for state in state_stats:
    print(f"{state['_id']}: {state['customer_count']} customers, {state['newsletter_subscribers']} newsletter subscribers")

Products by category:
Electronics: 48 products, avg price: $479.99, total stock: 1264
Sports: 40 products, avg price: $106.99, total stock: 1096
Kitchen: 32 products, avg price: $112.49, total stock: 616

Customers by state:
TX: 32 customers, 24 newsletter subscribers
CA: 24 customers, 8 newsletter subscribers
WA: 8 customers, 8 newsletter subscribers
NC: 8 customers, 8 newsletter subscribers
OH: 8 customers, 0 newsletter subscribers
AZ: 8 customers, 0 newsletter subscribers
PA: 8 customers, 8 newsletter subscribers
IL: 8 customers, 8 newsletter subscribers
NY: 8 customers, 8 newsletter subscribers
FL: 8 customers, 8 newsletter subscribers


In [40]:
# writing an aggregate to find the average rating for each product
pipeline = [
    {"$unwind": "$reviews"},
    {"$group": {
        "_id": "$product_id",
        "product_name": {"$first": "$name"},
        "avg_rating": {"$avg": "$reviews.rating"},
        "review_count": {"$sum": 1}
    }},
    {"$sort": {"avg_rating": -1}}
]

top_rated = list(products.aggregate(pipeline))
print("Top rated products:")
for product in top_rated[:5]:
    print(f"{product['product_name']}: {product['avg_rating']:.2f}")

Top rated products:
Gaming Mouse: 4.67
Laptop: 4.67
Microwave: 4.67
Running Shoes: 4.67
Tennis Racket: 4.67


In [41]:
#writing an aggregate to find the most expensive product in each category

topProductPipeline = [
    {"$group": {
        "_id": "$category",
        "productName":{"$first":"$name"},
        "price": {"$max":"$price"}
    }},
    {"$sort": {"price":-1}}
]

priceByCategory = list(products.aggregate(topProductPipeline))
print("The most expensive products are:")
for price in priceByCategory:
    print(f"{price['_id']} - {price['productName']}: ${price['price']:.2f}")

The most expensive products are:
Electronics - Wireless Headphones: $1299.99
Sports - Running Shoes: $199.99
Kitchen - Coffee Maker: $159.99


In [42]:
#writing an aggregate to find the customers by each state

statePipeline = [
    {"$group": {
        "_id":"$address.state",
        "customerCount" : {"$sum":1},
    }},
    {"$sort":{"customerCount":-1}}
]

customerByState = list(customers.aggregate(statePipeline))
print("Customers by State")
for customer in customerByState:
    print(f"{customer['_id']}: {customer['customerCount']} customers")

Customers by State
TX: 32 customers
CA: 24 customers
WA: 8 customers
NY: 8 customers
FL: 8 customers
PA: 8 customers
IL: 8 customers
AZ: 8 customers
OH: 8 customers
NC: 8 customers


In [43]:
# writing an aggregate to find total revenue by product category from orders
revenue_pipeline = [
    {"$unwind": "$items"},
    {"$lookup": {
        "from": "products",
        "localField": "items.product_id",
        "foreignField": "product_id",
        "as": "product_info"
    }},
    {"$unwind": "$product_info"}, 
    {"$group": {
        "_id": "$product_info.category",
        "revenue": {"$sum": {"$multiply": ["$items.price", "$items.quantity"]}}
    }},
    {"$sort": {"revenue": -1}}
]

revenue_by_category = list(orders.aggregate(revenue_pipeline))
print("Revenue by category:")
for category in revenue_by_category:
    print(f"{category['_id']}: ${category['revenue']:.2f}")

Revenue by category:
Electronics: $190715.52
Sports: $68793.60
Kitchen: $52474.88


In [44]:
#writing an aggregate to find the number of customers who have made more than 1 order

orders_pipeline = [
    {"$group": {
        "_id":"$customer_id",
        "order_count":{"$sum":1}
    }},
    {"$match": {"order_count":{"$gt":1}}
    }
]

multiple_orders = list(orders.aggregate(orders_pipeline))
print("Customers with Multiple Orders:")
for customer in multiple_orders:
    print(f"{customer['_id']}: {customer['order_count']}")

Customers with Multiple Orders:
C009: 8
C001: 8
C007: 8
C010: 8
C004: 8
C006: 8
C002: 8
C008: 8
C005: 8
C003: 8


3.6) The \\$unwind operator is used when working with nested arrays when creating aggregations. The unwind operator will take an array and create an output document for each element in an array, so if an array in an aggregation had 4 elements, the \\$unwind operator would make 4 separate documents so it is easier to access the information from the aggregate. This is useful because it is allows to perform calculations, calculate averages, and count array elements when working with array data. 

## Question 4

In [45]:
# using a function to convert MongoDB data to a pandas dataframe
import pandas as pd
from pymongo import MongoClient

# Convert MongoDB collection to DataFrame
def mongo_to_dataframe(collection, query=None):
    if query is None:
        query = {}
    cursor = collection.find(query)
    return pd.DataFrame(list(cursor))

# Get products data
products_df = mongo_to_dataframe(products)
print(f"Products DataFrame shape: {products_df.shape}")

Products DataFrame shape: (120, 10)


In [46]:
#examples of analyzing data using pandas
products_df = mongo_to_dataframe(products)
print("Product Analysis:")
print(f"Total products: {len(products_df)}")
print(f"Average price: ${products_df['price'].mean():.2f}")
print(f"Price range: ${products_df['price'].min():.2f} - ${products_df['price'].max():.2f}")

# Category analysis
category_analysis = products_df.groupby('category').agg({
    'price': ['mean', 'count'],
    'stock': 'sum'
}).round(2)
print("\nCategory Analysis:")
print(category_analysis)

# Calculate inventory value
products_df['inventory_value'] = products_df['price'] * products_df['stock']
inventory_by_category = products_df.groupby('category')['inventory_value'].sum().sort_values(ascending=False)
print("\nInventory Value by Category:")
print(inventory_by_category)

Product Analysis:
Total products: 120
Average price: $257.66
Price range: $24.99 - $1299.99

Category Analysis:
              price       stock
               mean count   sum
category                       
Electronics  479.99    48  1264
Kitchen      112.49    32   616
Sports       106.99    40  1096

Inventory Value by Category:
category
Electronics    367187.36
Sports          80189.04
Kitchen         65593.84
Name: inventory_value, dtype: float64


In [47]:
# how to handle nested data using pandas

# Work with customer data and nested addresses
customers_df = mongo_to_dataframe(customers)
print(f"Total customers: {len(customers_df)}")

# Extract state information from nested address
customers_df['state'] = customers_df['address'].apply(lambda x: x['state'])
state_analysis = customers_df['state'].value_counts()
print("\nCustomers by State:")
print(state_analysis)

# Analyze newsletter subscriptions
newsletter_subscribers = customers_df['preferences'].apply(lambda x: x['newsletter']).sum()
print(f"\nNewsletter subscribers: {newsletter_subscribers}")
print(f"Subscription rate: {newsletter_subscribers/len(customers_df)*100:.1f}%")

Total customers: 120

Customers by State:
state
TX    32
CA    24
NY     8
IL     8
AZ     8
PA     8
FL     8
OH     8
NC     8
WA     8
Name: count, dtype: int64

Newsletter subscribers: 80
Subscription rate: 66.7%


In [48]:
#analyzing order patterns by month

#creating the orders dataframe
orders_df = mongo_to_dataframe(orders)
items_df = pd.DataFrame([item for row in orders_df["items"] for item in row])
print(f"Total order items: {len(items_df)}")

#converting order_date to datetime field
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])

# calclulating the total sales and total orders
month_analysis = orders_df.groupby(orders_df['order_date'].dt.month).agg({
    "total_amount":"sum",
    "order_id" : "count"
})

print(month_analysis)

Total order items: 192
            total_amount  order_id
order_date                        
1                4559.52        24
2               23318.88        56


In [49]:
#finding the most popular product categories based on order quantities

# Join with product information to get categories
products_df = mongo_to_dataframe(products)
items_df = items_df.merge(
    products_df[["product_id", "category"]],
    on="product_id",
    how="left"
)

# Analyze items by category
category_analysis = items_df.groupby("category").agg({
    "quantity": "sum"
}).round(2)
print("\nOrder items by category:")
print(category_analysis)


Order items by category:
             quantity
category             
Electronics       448
Kitchen           512
Sports            640


4.6) Using MongoDB with Python data analysis libraries is beneficial because MongoDB allows us to query for the data we went and then utilize Python to convert the MongoDB database tables into dataframes. From there, we can use Python's array of libraries such as pandas, numpy and matplotlib to analyze, clean, visualize, and process the data for machine learning or other uses. 

## Question 5

In [52]:
# example of creating indexes and testing a query performance

# Create indexes for better performance
products.create_index("category")
products.create_index("price")

#creating a compound index
products.create_index([("category", 1), ("price", -1)]) 

# Test query performance
query = {"category": "Electronics"}
explain_result = db.command("explain", {"find": "products", "filter": query}, verbosity="executionStats")
print(f"Query execution time: {explain_result.get('executionStats', {}).get('executionTimeMillis', 'N/A')}ms")
print(f"Documents examined: {explain_result.get('executionStats', {}).get('totalDocsExamined', 'N/A')}")

Query execution time: 0ms
Documents examined: 48


In [68]:
#creating index and compound indexes for more advanced queries

products.create_index("tags")  # Multikey index for array fields
products.create_index("specifications.brand")  # Index on nested field
customers.create_index("email", unique=False)  # Unique index
orders.create_index([("customer_id", 1), ("order_date", -1)])  # Compound index

# Test different query patterns
queries_to_test = [
    {"category": "Electronics"},
    {"tags": "wireless"},
    {"specifications.brand": "Samsung"},
    {"price": {"$gt": 200, "$lt": 500}}
]

print("Query Performance Analysis:")
for i, query in enumerate(queries_to_test, 1):
    explain_result = db.command("explain", {"find": "products", "filter": query}, verbosity="executionStats")
    print(f"Query {i}: {query}")
    print(f"  Execution time: {explain_result.get('executionStats', {}).get('executionTimeMillis', 'N/A')}ms")
    print(f"  Documents examined: {explain_result.get('executionStats', {}).get('totalDocsExamined', 'N/A')}")
    print(f"  Documents returned: {explain_result.get('executionStats', {}).get('totalDocsReturned', 'N/A')}")
    print()

Query Performance Analysis:
Query 1: {'category': 'Electronics'}
  Execution time: 0ms
  Documents examined: 48
  Documents returned: N/A

Query 2: {'tags': 'wireless'}
  Execution time: 0ms
  Documents examined: 8
  Documents returned: N/A

Query 3: {'specifications.brand': 'Samsung'}
  Execution time: 0ms
  Documents examined: 0
  Documents returned: N/A

Query 4: {'price': {'$gt': 200, '$lt': 500}}
  Execution time: 0ms
  Documents examined: 16
  Documents returned: N/A



In [69]:
#creating indexes on nested fields
orders.create_index("shipping_address.state")
products.create_index("specifications.color")
customers.create_index("preferences.newsletter")

# Test different query patterns
queries_to_test = [
    {"shipping_address.state": "NY"},
    {"specifications.color":"Black"},
    {"preferences.newsletter": True}
]

print("Query Performance Analysis:")
for i, query in enumerate(queries_to_test, 1):
    explain_result = db.command("explain", {"find": "products", "filter": query}, verbosity="executionStats")
    print(f"Query {i}: {query}")
    print(f"  Execution time: {explain_result.get('executionStats', {}).get('executionTimeMillis', 'N/A')}ms")
    print(f"  Documents examined: {explain_result.get('executionStats', {}).get('totalDocsExamined', 'N/A')}")
    print(f"  Documents returned: {explain_result.get('executionStats', {}).get('totalDocsReturned', 'N/A')}")
    print()

Query Performance Analysis:
Query 1: {'shipping_address.state': 'NY'}
  Execution time: 0ms
  Documents examined: 120
  Documents returned: N/A

Query 2: {'specifications.color': 'Black'}
  Execution time: 0ms
  Documents examined: 8
  Documents returned: N/A

Query 3: {'preferences.newsletter': True}
  Execution time: 0ms
  Documents examined: 120
  Documents returned: N/A



In [71]:
#testing the performance of aggregation pipelines

# Check index usage statistics
db_stats = db.command("collStats", "products")
print("Collection Statistics:")
print(f"Total documents: {db_stats['count']}")
print(f"Average document size: {db_stats['avgObjSize']:.2f} bytes")
print(f"Total collection size: {db_stats['size']:.2f} bytes")
print(f"Index size: {db_stats['totalIndexSize']:.2f} bytes")

# List all indexes and their usage
indexes = list(products.list_indexes())
print(f"\nIndexes on products collection: {len(indexes)}")
for index in indexes:
    print(f"- {index['name']}: {index['key']}")

# Test aggregation performance
agg_pipeline = [
    {"$group": {"_id": "$category", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
agg_explain = db.command("explain", {"aggregate": "products", "pipeline": agg_pipeline, "cursor":{}}, verbosity="executionStats")
print(f"\nAggregation execution time: {agg_explain.get('executionStats', {}).get('executionTimeMillis', 'N/A')}ms")

Collection Statistics:
Total documents: 120
Average document size: 714.00 bytes
Total collection size: 85680.00 bytes
Index size: 208896.00 bytes

Indexes on products collection: 7
- _id_: SON([('_id', 1)])
- category_1: SON([('category', 1)])
- price_1: SON([('price', 1)])
- category_1_price_-1: SON([('category', 1), ('price', -1)])
- tags_1: SON([('tags', 1)])
- specifications.brand_1: SON([('specifications.brand', 1)])
- specifications.color_1: SON([('specifications.color', 1)])

Aggregation execution time: N/Ams


5.6) The key considerations for optimizing MongoDB performance in production is execution time, the number of documents MongoDB had to look at, how many documents were returned and the actual steps it took.

## Pledge

By submitting this work I hereby pledge that this is my own, personal work. I've acknowledged in the designated place at the top of this file all sources that I used to complete said work, including but not limited to: online resources, books, and electronic communications. I've noted all collaboration with fellow students and/or TA's. I did not copy or plagiarize another's work.

> As a Boilermaker pursuing academic excellence, I pledge to be honest and true in all that I do. Accountable together – We are Purdue.

https://www.purdue.edu/odos/osrr/honor-pledge/
